# TME1 - Recommendation Sociale

## Imports

In [1]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pickle as pkl

## Chargement des données

Les fonctions pour charger les bases Movie Lens 100k et Movie Lens 1M.
On récupère un dictionnaire pour les scores et un dictionnaire pour les dates.
Le première index de ces dictionnaires est l'identifiant de l'utilisateur, et le second les films notés.

In [2]:
def loadMovieLens(path='./data', dataFile='/u.data'):
    prefs={} # Un dictionnaire User > Item > Rating
    links = []
    for line in open(path+dataFile):
        (userId,movieId,rating,ts)=line.split('\t')
        prefs.setdefault(userId,{})
        prefs[userId][movieId]= float(rating)/5
    for line in open(path+'/u.links'):
        l = line[:-1].split('\t')
        source = l[0]
        if source in prefs.keys():
            for target in l[1:]:
                if target in prefs.keys():
                    links.append([source, target, 1])
    return prefs, links

In [3]:
def loadMovieLens1M(path='./data1m'):
    # Get movie titles
    movies={}
    for line in open(path+'/movies.dat'):
        id,title=line.split('::')[0:2]
        movies[id]=title
    # Load data
    prefs={}
    times={}
    for line in open(path+'/ratings.dat'):
        (user,movieid,rating,ts)=line.split('::')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
        times.setdefault(user,{})
        times[user][movies[movieid]]=float(ts)
    return prefs, times

## Représentations des données

Les matrices des scores Utilisateurs/Films sont des matrices de grandes dimensions mais sparses.
Afin de les manipuler efficacement, on emploiera 3 représentations différentes en même temps:
- Le dictionnaire des scores par utilisateurs: User > Item > Value
- Le dictionnaire des scores par films: Item > User > Value 
- La liste des triplets [User, Item, Value]

In [3]:
# Recupère une représentation des données sous la forme triplets [user, item, value] a partir d'un dictionnaire [User > item > value]
def getCouplesUsersItems(data):
    couples = []
    for u in data.keys():
        for i in data[u].keys():
            couples.append([u,i,data[u][i]])
    return couples

# Construit le dictionnaire des utilisateurs a partir des triplets [user, item, note]
def buildUsersDict(couples):
    dicUsers = {}
    for c in couples:
        if not c[0] in dicUsers.keys():
            dicUsers[c[0]] = {}
        dicUsers[c[0]][c[1]] = float(c[2])
    return dicUsers

# Construit le dictionnaire des objets a partir des triplets [user, item, note]
def buildItemsDict(couples):
    dicItems = {}
    for c in couples:
        if not c[1] in dicItems:
            dicItems[c[1]] = {}
        dicItems[c[1]][c[0]] = float(c[2])
    return dicItems

## Séparation des données en Train / Test

Pour pouvoir séparer les données en ensembles de Train et de Test, on utilisera la liste des triplets [User, Item, Scores].

In [4]:
# Split l'ensemble des triplets [user, item, note] en testProp% données de test et (1 - testProp) données de train
def splitTrainTest(couples,testProp):
    perm = np.random.permutation(couples)
    splitIndex = int(testProp * len(couples))
    return perm[splitIndex:], perm[:splitIndex]

# Modèles

On implémente ici les différents modèles. Les baselines prédisent simplement la note moyenne pour un utilisateur (ou pour un film) donné. Les modèles de factorisation matricielles tentent d'approximer les valeurs connues de la matrice des scores par un produit de deux matrices de dimensions inférieurs.

## Factorisation matricielle sans biais

On calcule les deux matrices P et Q tel que pour les exemples connus, PQ ~= X, où X est la matrice des scores.
Pour prédire, il suffit alors de lire dans la matrice PQ les nouveaux exemples.

In [5]:
class matrixFactorisation():
    def __init__(self, k, lambdaC=0.2, lambdaU=0.2, lambdaV=0.2, lambdaZ=0.2, eps=1e-5, maxIter=2000):
        self.k = k
        self.lambdaC = lambdaC
        self.lambdaU = lambdaU
        self.lambdaV = lambdaV
        self.lambdaZ = lambdaZ
        self.eps = eps
        self.maxIter = maxIter
        self.alternate = alternate #Pour l'optimisation alternée: 0 si non.
    def fit(self, tripletsUsersItems, tripletsLinks):
        self.u = {}
        self.v = {}
        self.z = {}
        self.loss = []
        #Choix du paramètre a optimisé en cas d'optimisation alternée
        for i in xrange(self.maxIter):
            lossUV = 0
            lossUZ = 0
            lossReg = 0
            for j in xrange(len(tripletsUsersItems)):
                # Ratings --------------------------------------------------------------------------------------------
                r = np.random.randint(len(tripletsUsersItems)) 
                user =   tripletsUsersItems[r][0]
                item =   tripletsUsersItems[r][1]
                rating = tripletsUsersItems[r][2]
                if not user in self.u:
                    self.u[user] = np.random.rand(1,self.k)
                if not item in self.v:
                    self.v[item] = np.random.rand(self.k,1)
                expUV = np.exp(self.u[user].dot(self.v[item])[0][0])
                logistiqueUV = (1.0/(1 + expUV))
                tmp = logistiqueUV - rating
                self.u[user] = self.u[user] - self.eps * tmp * expUV * (logistiqueUV **2) * self.v[item].transpose()
                self.v[item] = self.v[item] - self.eps * tmp * expUV * (logistiqueUV **2) * self.u[user].transpose()
                lossUV = lossUV + tmp*tmp/2. 
                # Links ---------------------------------------------------------------------------------------------
                r = np.random.randint(len(tripletsLinks))
                userSource = tripletsLinks[r][0]
                userTarget = tripletsLinks[r][1]
                linkScore  = tripletsLinks[r][2]
                if not userSource in self.u:
                    self.u[userSource] = np.random.rand(1,self.k)
                if not userTarget in self.z:
                    self.z[userTarget] = np.random.rand(self.k,1)
                expUZ = np.exp(self.u[userSource].dot(self.z[userTarget])[0][0])
                logistiqueUZ = (1.0/(1 + expUZ))
                tmp = logistiqueUZ - linkScore
                self.u[userSource] = self.u[userSource] - self.eps * tmp * expUZ * (logistiqueUZ **2) * self.z[userTarget].transpose()
                self.z[userTarget] = self.z[userTarget] - self.eps * tmp * expUZ * (logistiqueUZ **2) * self.u[userSource].transpose()
                lossUZ = lossUZ + tmp*tmp/2. 
                # Regularize  --------------------------------------------------------------------------------------
                ru = np.random.randint(len(self.u));
                rv = np.random.randint(len(self.v));
                rz = np.random.randint(len(self.z));
                self.u[ru] = self.u[ru] * (1 - self.lambdaU * self.eps)
                self.v[rv] = self.v[rv] * (1 - self.lambdaV * self.eps)
                self.z[rz] = self.z[rz] * (1 - self.lambdaZ * self.eps)
                lossReg = lossReg + np.sqrt((self.u[ru]**2).sum()) + np.sqrt((self.v[rv]**2).sum()) + np.sqrt((self.z[rz]**2).sum())
            self.loss.append([lossUV, lossUZ, lossReg])
            # Optimisation alternée
            if (i % 100 == 0):
                print i, loss / len(tripletsUsersItems)
    def predict(self, tripletsUsersItems):
        pred = np.zeros(lentripletsUsersItems)
        for ind,c in enumerate(tripletsUsersItems):
            pred[ind] = self.u[c[0]].dot(self.v[c[1]])[0][0]
        return pred

# Tests les données Movie Lens 100k

Les données Movie Lens 100k comprennent 100 000 scores données par 1000 utilisateurs sur 1700 films.

## Préparation des données

On extrait aléatoirement une portion (20%) des données pour constituer la base de test, et le reste sera utilisé en apprentissage.

Comme on ne souhaite ne pas évaluer les objets et les utilisateurs qui n'ont jamais été rencontré en apprentissage, on retire les couples correspondants de l'ensemble de test.

Reste ensuite à reconstruire les deux dictionnaires a partir de ces liste de couples.

In [8]:
# Chargement
data, timestamps = loadMovieLens()

# Récupérer la représentation en liste de triplets
couples = getCouplesUsersItems(data)

# La séparer en ensemble d'apprentissage et de test
trainCouples, testCouples = splitTrainTest(couples,.20)

# Reconstruire les dictionnaires pour l'ensemble d'apprentissage
trainUsers = buildUsersDict(trainCouples)
trainItems = buildItemsDict(trainCouples)

# Supprimer de l'ensemble de test les éléments inconnus en apprentissage
toDel = []
for i,c in enumerate(testCouples):
    if not c[0] in trainUsers:
        toDel.append(i)
    elif not c[1] in trainItems:
        toDel.append(i)
testCouples = np.delete(testCouples, toDel, 0)

# Reconstruire les dictionnaires pour l'ensemble de test
testUsers  = buildUsersDict(testCouples)
testItems  = buildItemsDict(testCouples)

# Récupérer les vecteurs des temps
#nbins = 5
#times = getTimeBins(couples, timestamps, nbins)
#trainTimes = getTimeBins(trainCouples, timestamps, nbins)
#testTimes = getTimeBins(testCouples, timestamps, nbins)

# taille des données
#print len(trainUsers), len(testUsers)
#print len(trainItems), len(testItems)

In [8]:
# Chargement
data, timestamps = loadMovieLens()

# Récupérer la représentation en liste de triplets
couples = getCouplesUsersItems(data)

# La séparer en ensemble d'apprentissage et de test
trainCouples, testCouples = splitTrainTest(couples,.20)

# Reconstruire les dictionnaires pour l'ensemble d'apprentissage
trainUsers = buildUsersDict(trainCouples)
trainItems = buildItemsDict(trainCouples)

# Supprimer de l'ensemble de test les éléments inconnus en apprentissage
toDel = []
for i,c in enumerate(testCouples):
    if not c[0] in trainUsers:
        toDel.append(i)
    elif not c[1] in trainItems:
        toDel.append(i)
testCouples = np.delete(testCouples, toDel, 0)

# Reconstruire les dictionnaires pour l'ensemble de test
testUsers  = buildUsersDict(testCouples)
testItems  = buildItemsDict(testCouples)

# Récupérer les vecteurs des temps
#nbins = 5
#times = getTimeBins(couples, timestamps, nbins)
#trainTimes = getTimeBins(trainCouples, timestamps, nbins)
#testTimes = getTimeBins(testCouples, timestamps, nbins)

# taille des données
#print len(trainUsers), len(testUsers)
#print len(trainItems), len(testItems)

## Factorisation matricielle sans biais

Après apprentissage, la factorisation matricielle donne une erreur moyenne en test de 0.9
Il est meilleur que les baselines de 0.1 point.
On note que le loss en apprentissage est de 0.84. 
Il est possible que l'on obtienne de meilleurs score de généralisation en test en augmentant la régularisation mais au vu du score actuel en apprentissage, le gain devrait rester assez faible.

In [8]:
nIter = 1000

print 'lambda = 0.2 -- k = 10'
model12 = matrixFactorisation(10, lambd=0.2, maxIter=nIter, alternate=0)
model12.fit(trainUsers, trainItems, trainCouples)

print ''
print 'lambda = 0.2 -- k = 20'
model22 = matrixFactorisation(20, lambd=0.2, maxIter=nIter, alternate=0)
model22.fit(trainUsers, trainItems, trainCouples)

print ''
print 'lambda = 0.2 -- k = 30'
model32 = matrixFactorisation(30, lambd=0.2, maxIter=nIter, alternate=0)
model32.fit(trainUsers, trainItems, trainCouples)

print ''
print 'lambda = 0.3 -- k = 10'
model13 = matrixFactorisation(10, lambd=0.3, maxIter=nIter, alternate=0)
model13.fit(trainUsers, trainItems, trainCouples)

print ''
print 'lambda = 0.3 -- k = 20'
model23 = matrixFactorisation(20, lambd=0.3, maxIter=nIter, alternate=0)
model23.fit(trainUsers, trainItems, trainCouples)

print ''
print 'lambda = 0.3 -- k = 30'
model33 = matrixFactorisation(30, lambd=0.3, maxIter=nIter, alternate=0)
model33.fit(trainUsers, trainItems, trainCouples)

print ''
print 'lambda = 0.5 -- k = 10'
model15 = matrixFactorisation(10, lambd=0.5, maxIter=nIter, alternate=0)
model15.fit(trainUsers, trainItems, trainCouples)

print ''
print 'lambda = 0.5 -- k = 20'
model25 = matrixFactorisation(20, lambd=0.5, maxIter=nIter, alternate=0)
model25.fit(trainUsers, trainItems, trainCouples)

print ''
print 'lambda = 0.5 -- k = 30b'
model35 = matrixFactorisation(30, lambd=0.5, maxIter=nIter, alternate=0)
model35.fit(trainUsers, trainItems, trainCouples)

lambda = 0.2 -- k = 10
0 2.75190583881
100 1.27670001304
200 1.05293615912
300 0.969963886409
400 0.943293214479
500 0.90723716787
600 0.888669343455
700 0.869424519454
800 0.869385926297
900 0.865527016505

lambda = 0.2 -- k = 20
0 4.35806281114
100 1.34847073088
200 1.13393452347
300 1.04544534464
400 0.975056903677
500 0.950182024409
600 0.920791608705
700 0.902388641713
800 0.881111783452
900 0.871287402788

lambda = 0.2 -- k = 30
0 17.7533425698
100 1.9512292745
200 1.46638301238
300 1.25946252452
400 1.15010952968
500 1.07464404341
600 1.01122887199
700 0.973330111398
800 0.930121793995
900 0.92312504415

lambda = 0.3 -- k = 10
0 2.87934713069
100 1.31697542334
200 1.07979728159
300 0.983712075982
400 0.952669019333
500 0.917599070791
600 0.904778274079
700 0.886987595752
800 0.877493241739
900 0.876678662032

lambda = 0.3 -- k = 20
0 4.31536424231
100 1.32108225123
200 1.13047045394
300 1.04004580885
400 0.980687503694
500 0.948052780935
600 0.923878956787
700 0.897181713626
800

In [9]:
print 'lambda = 0.2 -- k = 10'
pred = model12.predict(testCouples)
print "Erreur de validation:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

print ''
print 'lambda = 0.2 -- k = 20'
pred = model22.predict(testCouples)
print "Erreur de validation:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

print ''
print 'lambda = 0.2 -- k = 30'
pred = model32.predict(testCouples)
print "Erreur de validation:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

print ''
print 'lambda = 0.3 -- k = 10'
pred = model13.predict(testCouples)
print "Erreur de validation:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

print ''
print 'lambda = 0.3 -- k = 20'
pred = model23.predict(testCouples)
print "Erreur de validation:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

print ''
print 'lambda = 0.3 -- k = 30'
pred = model33.predict(testCouples)
print "Erreur de validation:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

print ''
print 'lambda = 0.5 -- k = 10'
pred = model15.predict(testCouples)
print "Erreur de validation:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

print ''
print 'lambda = 0.5 -- k = 20'
pred = model25.predict(testCouples)
print "Erreur de validation:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

print ''
print 'lambda = 0.5 -- k = 30'
pred = model35.predict(testCouples)
print "Erreur de validation:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

lambda = 0.2 -- k = 10
Erreur de validation: 0.943174131203

lambda = 0.2 -- k = 20
Erreur de validation: 0.986211376994

lambda = 0.2 -- k = 30
Erreur de validation: 1.06369432066

lambda = 0.3 -- k = 10
Erreur de validation: 0.954552418356

lambda = 0.3 -- k = 20
Erreur de validation: 0.982911937276

lambda = 0.3 -- k = 30
Erreur de validation: 1.05734143386

lambda = 0.5 -- k = 10
Erreur de validation: 0.986438937944

lambda = 0.5 -- k = 20
Erreur de validation: 1.00782308867

lambda = 0.5 -- k = 30
Erreur de validation: 1.07272716099


In [17]:
nIter = 1000

print 'lambda = 0.1 -- k = 5'
model51 = matrixFactorisation(5, lambd=0.1, maxIter=nIter, alternate=0)
model51.fit(trainUsers, trainItems, trainCouples)

print ''
print 'lambda = 0.1 -- k = 10'
model11 = matrixFactorisation(10, lambd=0.1, maxIter=nIter, alternate=0)
model11.fit(trainUsers, trainItems, trainCouples)

print ''
print 'lambda = 0.1 -- k = 20'
model21 = matrixFactorisation(20, lambd=0.1, maxIter=nIter, alternate=0)
model21.fit(trainUsers, trainItems, trainCouples)

lambda = 0.1 -- k = 5
0 6.66796103276
100 2.16533111433
200 1.39525666024
300 1.1673575132
400 1.05631970404
500 0.995184225778
600 0.946183494468
700 0.931376174196
800 0.909948933309
900 0.89410511451

lambda = 0.1 -- k = 10
0 2.72804780588
100 1.24562457796
200 1.06159033004
300 0.975549749359
400 0.93749079135
500 0.902570092986
600 0.885234677137
700 0.866488510075
800 0.859612625993
900 0.855580345388

lambda = 0.1 -- k = 20
0 4.61384248304
100 1.36043731792
200 1.14179483971
300 1.05519015956
400 1.00207536134
500 0.948788341113
600 0.930152890945
700 0.896155884921
800 0.882564699252
900 0.861024616148


In [11]:
nIter = 1000

print 'lambda = 0.05 -- k = 5'
model55 = matrixFactorisation(5, lambd=0.1, maxIter=nIter, alternate=0)
model55.fit(trainUsers, trainItems, trainCouples)

print ''
print 'lambda = 0.05 -- k = 10'
model15 = matrixFactorisation(10, lambd=0.1, maxIter=nIter, alternate=0)
model15.fit(trainUsers, trainItems, trainCouples)

print ''
print 'lambda = 0.05 -- k = 20'
model25 = matrixFactorisation(20, lambd=0.1, maxIter=nIter, alternate=0)
model25.fit(trainUsers, trainItems, trainCouples)

lambda = 0.05 -- k = 5
0 6.77769404477
100 2.15509847904
200 1.37105808991
300 1.15232790068
400 1.05174831243
500 0.988557112616
600 0.960754916958
700 0.924799100186
800 0.905153306735
900 0.891524703968

lambda = 0.05 -- k = 10
0 2.67692891284
100 1.24229347112
200 1.04394395163
300 0.972019387925
400 0.923460427147
500 0.902979809161
600 0.888171858253
700 0.871944998967
800 0.859733199529
900 0.862137284575

lambda = 0.05 -- k = 20
0 4.13608289007
100 1.34636260249
200 1.14267168158
300 1.04161666263
400 0.991045419567
500 0.951876564253
600 0.928072365333
700 0.906086161118
800 0.878199020996
900 0.871792798825


In [18]:
print 'lambda = 0.1 -- k = 5'
pred = model51.predict(testCouples)
print "Erreur de validation:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

print ''
print 'lambda = 0.1 -- k = 10'
pred = model11.predict(testCouples)
print "Erreur de validation:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

print ''
print 'lambda = 0.1 -- k = 20'
pred = model21.predict(testCouples)
print "Erreur de validation:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

print ''
print 'lambda = 0.05 -- k = 5'
pred = model55.predict(testCouples)
print "Erreur de validation:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

print ''
print 'lambda = 0.05 -- k = 10'
pred = model15.predict(testCouples)
print "Erreur de validation:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

print ''
print 'lambda = 0.05 -- k = 20'
pred = model25.predict(testCouples)
print "Erreur de validation:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

lambda = 0.1 -- k = 5
Erreur de validation: 0.979172611826

lambda = 0.1 -- k = 10
Erreur de validation: 0.952456385191

lambda = 0.1 -- k = 20
Erreur de validation: 1.00013579314

lambda = 0.05 -- k = 5
Erreur de validation: 0.973878095195

lambda = 0.05 -- k = 10
Erreur de validation: 0.954999388066

lambda = 0.05 -- k = 20
Erreur de validation: 0.997131262701


In [12]:
nIter = 1000

lambdas = [0.05, 0.1, 0.2, 0.3]
ks = [5,10,20,30]

bestErrNoBias = 10

for l in lambdas:
    for k in ks:
        print 'lambda=', l, " k=", k
        model = matrixFactorisation(k, lambd=l, maxIter=nIter, alternate=0)
        model.fit(trainUsers, trainItems, trainCouples)
        pred = model.predict(testCouples)
        err = ((pred - np.array(testCouples[:,2], float)) ** 2).mean()
        if err < bestErrNoBias:
            bestErrNoBias = err
            bestLambdaNoBias = l
            bestKNoBias = k
        print "Erreur de validation:", err
        print ""

print 'Meilleur paramètres trouvés (avec une erreur en validation de ', bestErrNoBias, '):'
print "lambda=",bestLambdaNoBias
print "k=",bestKNoBias

lambda= 0.05  k= 5
0 6.81339039146
100 2.1739072049
200 1.39567802908
300 1.16649311097
400 1.06155437606
500 0.996255640528
600 0.953648781237
700 0.930414317154
800 0.902901434743
900 0.893661283824
Erreur de validation: 0.968927919066

lambda= 0.05  k= 10
0 2.74602702444
100 1.266597751
200 1.04408844543
300 0.966827294684
400 0.932129089865
500 0.908533609597
600 0.892018218519
700 0.875784940917
800 0.859686622315
900 0.850167283353
Erreur de validation: 0.947914689664

lambda= 0.05  k= 20
0 4.19968687518
100 1.33949743908
200 1.13764166872
300 1.04435499624
400 0.974194053417
500 0.951005587689
600 0.908900398868
700 0.900403284349
800 0.87974675115
900 0.857951996135
Erreur de validation: 0.991305129105

lambda= 0.05  k= 30
0 17.7907707054
100 1.97433063129
200 1.46936591673
300 1.2818344391
400 1.17121300044
500 1.08603204549
600 1.01620713887
700 0.978029524988
800 0.936991979119
900 0.914175775133
Erreur de validation: 1.08278510207

lambda= 0.1  k= 5
0 6.67735743984
100 2.19

In [ ]:
nIter = 1000

lambdas = [0.05, 0.1, 0.2, 0.3]
ks = [5,10,20,30]

bestErr = 10

for l in lambdas:
    for k in ks:
        print 'lambda=', l, " k=", k
        model = matrixFactorisationBiais(k, lambd=l, maxIter=nIter, alternate=0)
        model.fit(trainUsers, trainItems, trainCouples)
        pred = model25.predict(testCouples)
        err = ((pred - np.array(testCouples[:,2], float)) ** 2).mean()
        if err < bestErr:
            bestErr = err
            bestLambda = l
            bestK = k
        print "Erreur de validation:", err
        print ""

print 'Meilleur paramètres trouvés (avec une erreur en validation de ', bestErr, '):'
print "lambda=",bestLambda
print "k=",bestK

# Experiences sur les données Movie Lens 1M

Le dataset Movie Lens 1M contient 1 million d'entrées, données par 6000 utiilisateurs sur 4000 films.

Une remarque que l'on peut déjà faire est que la matrice des scores est "moins sparse" que celle issue de la base 100k.

En effet, pour la base 100k on a 100000 notes pour une matrice 1000x1700, soit un remplissage de 17% de la matrice.

Pour la base 1M, on a 1 000 000 de notes pour une matrice 6000x4000, soit 24% de remplissage.

## Préparation des données

In [ ]:
# Chargement
data, timestamps = loadMovieLens1M()

# Récupérer la représentation en liste de triplets
couples = getCouplesUsersItems(data)

# Séparer en ensemble d'apprentissage et de test
trainCouples, testCouples = splitTrainTest(couples,.20)

# Reconstruire les dictionnaires pour l'ensemble d'apprentissage
trainUsers = buildUsersDict(trainCouples)
trainItems = buildItemsDict(trainCouples)

# Supprimer de l'ensemble de test les éléments inconnus en apprentissage
toDel = []
for i,c in enumerate(testCouples):
    if not c[0] in trainUsers:
        toDel.append(i)
    elif not c[1] in trainItems:
        toDel.append(i)
testCouples = np.delete(testCouples, toDel, 0)

# Reconstruire les dictionnaires pour l'ensemble de test
testUsers  = buildUsersDict(testCouples)
testItems  = buildItemsDict(testCouples)

# taille des données
#print len(trainUsers), len(testUsers)
#print len(trainItems), len(testItems)

## Factorisation Matricielle

Avec un score en apprentissage de 0.82 et de généralisation de 0.85, on peut estimer que le paramètre de régularisation choisi (lambda = 0.2) est raisonnable pour ce problème. La factorisation matricielle est aussi meilleure que celle de la base 100k, probablement parceque la matrice d'apprentissage est moins sparse.

In [ ]:
model8 = matrixFactorisation(10, alternate=0, maxIter=1000)
model8.fit(trainUsers, trainItems, trainCouples)

In [ ]:
plt.figure()
plt.plot(model8.loss)
plt.show()

In [ ]:
pred = model8.predict(testCouples)
print ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

# Conclusion

Dans ce travail, nous avons implémenté des modèles de filtrage collaboratif que nous avons ensuite chercher à évaluer.
Si on a observé qu'un modèle simple de factorisation matricielle pouvait obtenir des résultats concluants, meilleurs qu'une baseline naïve, nous n'avons pu démontré d'intérêt pratiques des modèles avec biais.

Cependant, nous avons observés que si nos modèles sans biais avaient des scores similaires en apprentissage qu'en généralisation, ce n'était pas le cas de nos modèles avec biais qui obtiennent de bien meilleurs scores en apprentissage qui ne se traduisent pas en test. On peut en déduire que nous n'avons pas déterminé les bons hyperparamètres pour permettre une bonne généralisation, et que vraisemblablement on peut encore augmenter le score en généralisation de nos modèles avec biais.

Enfin, nous n'avons eu le temps ni d'évaluer les modèles avec biais temporel, ni l'influence de la dimension de factorisation matricielle, que l'on peut aussi voir comme une forme de régularisation.

# tSNE

In [2]:
from scipy.spatial import distance

class tSNE():
    def __init__(self,perp, nIter, lr, moment, dim=2):
        self.perp = perp # entre 5 et 50
        self.nIter = nIter
        self.lr = lr
        self.moment = moment
        self.dim = dim 
    def fit(self,data):
        nEx = np.shape(data)[0]
        # Matrice des distances de ||xi - xj||² #
        normx = np.sum(data**2,1)
        normx = np.reshape(normx, (1, nEx))
        distancex = normx + normx.T - 2 * data.dot(data.T)
        # Calcul des sigma ---------------------------------------------------------------#
        lperp = np.log2(self.perp)
        # initialisation bornes pour la recherche dichotomique #
        sup = np.ones((nEx,1)) * np.max(distancex)
        inf = np.zeros((nEx,1))
        self.sigma = (sup + inf) / 2.
        # recherche dichotomique #
        stop = False
        while not stop:
            # Calculer la matrice des p(i|j)
            self.pcond = np.exp(-distancex / (2. * (self.sigma**2)))
            self.pcond = self.pcond / np.sum(self.pcond - np.eye(nEx),1).reshape(nEx,1)
            # Calculer l'entropie de p(i|j)
            entropy = - np.sum(self.pcond * np.log2(self.pcond), 0)
            # Mise a jour des bornes
              # Si il faut augmenter sigma
            up = entropy < lperp 
            inf[up,0] = self.sigma[up,0]
              # Si il faut baisser sigma
            down = entropy > lperp 
            sup[down,0] = self.sigma[down,0]
            # Mise a jour de sigma et condition d'arrêt
            old = self.sigma
            self.sigma = ((sup + inf) / 2.)
            if np.max(np.abs(old - self.sigma)) < 1e-5:
                stop = True
                print np.exp(entropy)
                print self.sigma.T  
        #--------------------------------------------------------------------------#
        #initialiser y
        self.embeddings = np.zeros((self.nIter+2, nEx, self.dim))
        self.embeddings[1] = np.random.randn(nEx, self.dim) * 1e-4
        #--------------------------------------------------------------------------#
        # p(ij)
        self.pij = (self.pcond + self.pcond.T) / (2.*nEx)
        np.fill_diagonal(self.pij, 0)
        # Descente de Gradient
        loss = []
        for t in xrange(1,self.nIter+1):
            # Matrice des distances 
            normy = np.sum((self.embeddings[t]**2),1)
            normy = np.reshape(normy, (1, nEx))
            distancey = normy + normy.T - 2 * self.embeddings[t].dot(self.embeddings[t].T)
            # q(ij)
            # self.qij = (distancey.sum() + nEx*(nEx-1)) / (1 + distancey)
            # np.fill_diagonal(self.qij, 0)
            self.qij = 1 / (1 + distancey)
            np.fill_diagonal(self.qij, 0)
            self.qij = self.qij / self.qij.sum()
            # Descente de gradient
            yt = self.embeddings[t]
            tmpgrad = 4 * ((self.pij - self.qij) / (1 + distancey)).reshape(nEx, nEx,1)
            for i in range(nEx):
                dy = (tmpgrad[i] * (yt[i]-yt)).sum(0)
                self.embeddings[t+1][i] = yt[i] - self.lr * dy + self.moment * (yt[i] - self.embeddings[t-1,i])
            l = stats.entropy(self.pij, self.qij, 2).mean()
            loss.append(l)
            print t,l

# Digits Dataset

In [3]:
from sklearn import datasets
data = datasets.load_digits()

In [6]:
model = tSNE(30,100,1000,0)
model.fit(data.data)

[ 135.20805638  135.24193477   26.39898988 ...,  135.21719547  135.21895758
   74.20709055]
[[  4.91911469   8.38679509  11.59179135 ...,   9.254232     9.27069259
   11.59179135]]
1 4.34669798947
2 4.34669796982
3 4.34669788061
4 4.34669731198
5 4.34669324971
6 4.34666216772
7 4.3464139267
8 4.34438239594
9 4.32794268117
10 4.21765430868
11 3.85737933314
12 3.46447047565
13 3.20934723581
14 3.02972953624
15 2.89821339501
16 2.80141776817
17 2.71747409936
18 2.65586093211
19 2.59158189631
20 2.54661100407
21 2.49834755728
22 2.46083330985
23 2.41803733743
24 2.38521233262
25 2.34711267745
26 2.31701824169
27 2.28360985272
28 2.25627299233
29 2.22753337538
30 2.20269278029
31 2.17797985011
32 2.15505283928
33 2.13365323189
34 2.11223531038
35 2.09350121883
36 2.07342910919
37 2.056847753
38 2.0381173177
39 2.02324748539
40 2.00587710909
41 1.99237702188
42 1.97633414146
43 1.96397209545
44 1.949135865
45 1.93774593696
46 1.92394461185
47 1.91338591709
48 1.90044605735
49 1.89059351781
5

In [5]:
t = np.shape(model.embeddings)[0] -1
plt.figure()
plt.plot(model.embeddings[t,:,0][data.target == 0], model.embeddings[t,:,1][data.target == 0], 'o', color="blue")
plt.plot(model.embeddings[t,:,0][data.target == 1], model.embeddings[t,:,1][data.target == 1], 'o', color="red")
plt.plot(model.embeddings[t,:,0][data.target == 2], model.embeddings[t,:,1][data.target == 2], 'o', color="cyan")
plt.plot(model.embeddings[t,:,0][data.target == 3], model.embeddings[t,:,1][data.target == 3], 'o', color="magenta")
plt.plot(model.embeddings[t,:,0][data.target == 4], model.embeddings[t,:,1][data.target == 4], 'o', color="yellow")
plt.plot(model.embeddings[t,:,0][data.target == 5], model.embeddings[t,:,1][data.target == 5], 'o', color="black")
plt.plot(model.embeddings[t,:,0][data.target == 6], model.embeddings[t,:,1][data.target == 6], 'o', color="white")
plt.plot(model.embeddings[t,:,0][data.target == 7], model.embeddings[t,:,1][data.target == 7], 'o', color=(0.5, 0.5, 0))
plt.plot(model.embeddings[t,:,0][data.target == 8], model.embeddings[t,:,1][data.target == 8], 'o', color=(0,0.5,0.5))
plt.plot(model.embeddings[t,:,0][data.target == 9], model.embeddings[t,:,1][data.target == 9], 'o', color=(0.5,0,0.5))
plt.show()

# Movie Lens 100k

In [ ]:
import pickle as pkl
fichier = open("./model.p")
reco = pkl.load(fichier)
fichier.close()

In [ ]:
movies = np.zeros((len(reco.q),10))
titles = []
for i,q in enumerate(reco.q.keys()):
    movies[i] = np.reshape(reco.q[q],10,1)
    titles.append(q)

In [ ]:
model = tSNE(30,1000,1e3,0)
model.fit(movies)

In [ ]:
plt.figure()
plt.plot(model.embeddings[11,:,0],model.embeddings[11,:,1], 'o', color="blue")


#y=[2.56422, 3.77284,3.52623,3.51468,3.02199]
#z=[0.15, 0.3, 0.45, 0.6, 0.75]
#n=[58,651,393,203,123]

#fig, ax = plt.subplots()
#ax.scatter(z, y)

for i, txt in enumerate(titles):
    if (np.random.rand() > .9):
        plt.annotate(txt.decode('latin-1'), model.embeddings[11,i])

plt.show()